In [1]:
!echo "{\"username\":\"gabeisaacs\",\"key\":\"a50fa86477b5f2f6c6d3d1e94969d1d7\"}"> kaggle.json
!sudo mkdir -p ~/.kaggle
!sudo cp /content/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c dogs-vs-cats
!unzip /content/dogs-vs-cats.zip

 99% 806M/812M [00:05<00:00, 154MB/s]
100% 812M/812M [00:05<00:00, 150MB/s]
Archive:  /content/dogs-vs-cats.zip
  inflating: sampleSubmission.csv    
  inflating: test1.zip               
  inflating: train.zip               


In [2]:
import os
# create directories
dataset_home = 'dataset_dogs_vs_cats/'
subdirs = ['train/', 'test/']
for subdir in subdirs:
 # create label subdirectories
 labeldirs = ['dogs/', 'cats/']
 for labldir in labeldirs:
  newdir = dataset_home + subdir + labldir
  os.makedirs(newdir, exist_ok=True)

import zipfile
import os

def unzip_file(zip_file, extract_to):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Example usage:
zip_file = 'train.zip'
extract_to = 'dataset_dogs_vs_cats/dataTrain'

# Make sure the extraction directory exists or create it if necessary
if not os.path.exists(extract_to):
    os.makedirs(extract_to)

unzip_file(zip_file, extract_to)

zip_file = 'test1.zip'
extract_to = 'dataset_dogs_vs_cats/dataTest'
unzip_file(zip_file, extract_to)

In [3]:
import os
import shutil
import random
# seed random number generator
random.seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.25

src_directory = 'dataset_dogs_vs_cats/dataTrain/train'
for file in os.listdir(src_directory):
    src = os.path.join(src_directory, file)
    dst_dir = 'dataset_dogs_vs_cats/train/'
    if random.random() < val_ratio:
      dst_dir = 'dataset_dogs_vs_cats/test/'
    if file.startswith('cat'):
        dst = os.path.join(dst_dir, 'cats', file)
        shutil.copyfile(src, dst)
    elif file.startswith('dog'):
        dst = os.path.join(dst_dir, 'dogs', file)
        shutil.copyfile(src, dst)

directory = 'dataset_dogs_vs_cats/test/cats'
files = os.listdir(directory)
total_files = len(files)
print("Total number of files:", total_files)

Total number of files: 3181


In [ ]:
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

# Define CNN model
def define_model():
    # Load model
    model = VGG16(include_top=False, input_shape=(224, 224, 3))

    # Mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False

    # Add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)

    # Define new model
    model = Model(inputs=model.inputs, outputs=output)

    # Compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Plot diagnostic learning curves
def summarize_diagnostics(history):
    # Plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')

    # Plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')

    # Save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()

# Run the test harness for evaluating a model
def run_test_harness():
    # Define model
    model = define_model()

    # Create data generator
    datagen = ImageDataGenerator(featurewise_center=True)

    # Specify ImageNet mean values for centering
    datagen.mean = [123.68, 116.779, 103.939]

    # Prepare iterator
    train_it = datagen.flow_from_directory('dataset_dogs_vs_cats/train/',
                                           class_mode='binary', batch_size=64, target_size=(224, 224))
    test_it = datagen.flow_from_directory('dataset_dogs_vs_cats/test/',
                                          class_mode='binary', batch_size=64, target_size=(224, 224))

    # Fit model
    history = model.fit_generator(train_it, steps_per_epoch=len(train_it),validation_data=test_it, validation_steps=len(test_it), epochs=2, verbose=1)

    # Evaluate model
    _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
    print('> %.3f' % (acc * 100.0))

    # Learning curves
    summarize_diagnostics(history)

# Entry point, run the test harness
run_test_harness()


58889256/58889256 [==============================] - 0s 0us/step


Found 18697 images belonging to 2 classes.
Found 6303 images belonging to 2 classes.


<ipython-input-4-c1e627d9ca77>:70: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_it, steps_per_epoch=len(train_it),validation_data=test_it, validation_steps=len(test_it), epochs=2, verbose=1)


Epoch 1/2
293/293 [==============================] - ETA: 0s - loss: 0.7645 - accuracy: 0.9350 